<img src="saverlife.png" width="400">

In [22]:
import ipywidgets as widgets
from fastai.text.all import *
import urllib.request
from os.path import exists
import requests

In [23]:
widgets.HTML('<H1><Center>Chat Genie Demo</h1><H2>Write a question and hit enter')

HTML(value='<H1><Center>Chat Genie Demo</h1><H2>Write a question and hit enter')

In [3]:
MODEL_URL = "https://www.dropbox.com/s/49f9shqvc9tj13b/model_augmented_july6.pkl?dl=1"
if not exists('model_augmented_july6.pkl'):
    urllib.request.urlretrieve(MODEL_URL, "model_augmented_july6.pkl")

learn = load_learner("model_augmented_july6.pkl")
sdf = pd.read_pickle('segdf_augmented_jul6.pkl')

In [4]:
textbox = widgets.Text(
    value='How do I save more money?',
    placeholder='Write your question here!',
    description='Question:',
    disabled=False,
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

In [15]:
class preds:
    question = None
    DF = None
    email = ''
    num_results = 13
    results = []
    options = ['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"]

    
def get_predictions(text):
    pred = learn.predict(text)
    df = pd.DataFrame(pred[2], columns = ['pct'])
    df['label_idx'] = df.index
    df = df.sort_values('pct', ascending=False).iloc[:50]
    url_df = pd.DataFrame(learn.dls.vocab[1])
    url_df['label_idx'] = url_df.index
    df = df.merge(url_df)
    df.columns = ['pct', 'idx', 'url_segment']
    df = df.merge(sdf)
    df['score'] = df['pct'] * df['multiplier']
    df = df.sort_values('score', ascending=False).reset_index()
    preds.DF = df
    preds.question = text

    
def write_db(data):
    headers = {'Content-type': 'application/json'}
    if preds.email:
        data = data[:-1] + ',\"email\":\"' + preds.email + '"}'
    response = requests.post(
        'https://sl2-datascience-api.herokuapp.com/rate_bot_response/',
        headers=headers, data=data)
    return(response)
    
    
def store_rating(Rating, result):
    if Rating != 'Did that answer your question?':
        question = preds.question.replace("'", "''")
        dic = {'Yes': 1, 'Somewhat': 2, 'Not at all': 3}
        url = preds.DF.iloc[result]['url']
        segment = int(preds.DF.iloc[result]['segment'])
        data = '{\"question\":\"' + question + \
            '\",\"url\":\"' + url + \
            '\",\"segment\":\"' + str(segment) + \
            '\",\"rating\":\"' + Rating + \
            '\",\"result\":\"' + str(result+1) + '"}'
#         if preds.email:
#             data = data[:-1] + ',\"email\":\"' + preds.email + '"}'
        response = write_db(data)
        if response.json():
            print(f'\033[91m There was an error storing your rating. Did you use "quotes"?')
            print(response.json())
        else: print(f'\033[36m Thank You for rating result #{result+1}!')
        response.close()
    else: print('')
        
    
def setup_text():
    preds.results = []
    for n in range(preds.num_results):
        preds.results += [widgets.HTML(value=f"""
            <hr><b><h2 text style ="line-height:1.5;"><center>{preds.DF.headline[n]}</center></h2>
            <h3 text style="color:GREEN; line-height:1.5;">{preds.DF.subheading[n]}</b></h3>
            <p text style ="line-height:1.5;">{preds.DF.text[n]}</p>
            <text style="color:BLUE;"><a href="{preds.DF.url[n]}" target="_blank">
            Read the article on the Saverlife blog</a>""")]
    
    
def display_text():
    display(widgets.HTML(f'''<h4 text style ="line-height:1.5;"> Top {preds.num_results}
        Results for "{preds.question}" '''))
#     </h4> <p text style ="line-height:1.5;">Please rate your results below
#     widgets.interact(set_email, email='example@saverlife.org')
    for n in range(preds.num_results):
        display(preds.results[n])
        widgets.interact(store_rating, Rating=preds.options,
                         result = widgets.fixed(n));

def store_search(n):
    data = '{\"question\":\"' + preds.question.replace("'", "''") + \
        '\",\"url\":\"NA' + \
        '\",\"segment\":\"0' + \
        '\",\"rating\":\"None' + \
        '\",\"result\":\"' + str(n) + '"}'
    write_db(data)
        
        
def run_predictions(text, n):
    preds.num_results = n
    get_predictions(text)
    store_search(n)
    setup_text()
    display_text()
    
    
def set_email(email):
    if email != 'example@saverlife.org': preds.email = email

HTML(value='<H1><Center>Chat Genie Demo</h1><H2>Write a question and hit enter')

In [17]:
widgets.HBox([widgets.VBox([textbox])])

HTML(value='<H4>Select the number of results and enter your email (optional)')

In [32]:
display(widgets.HTML('''<H4>Select the number of results, enter your email
    (optional), and rate your responses below'''))
slider = widgets.IntSlider(
    description='Responses', max=25, min=1, value=5, continuous_update=False)
display(slider)
em = widgets.interact(set_email, email='example@saverlife.org')

HTML(value='<H4>Select the number of results, enter your email\n    (optional), and rate your responses below'…

IntSlider(value=5, continuous_update=False, description='Responses', max=25, min=1)

interactive(children=(Text(value='example@saverlife.org', description='email'), Output()), _dom_classes=('widg…

In [33]:
iw = widgets.interactive_output(run_predictions, {'text': textbox, 'n': slider})
z = widgets.HBox([widgets.VBox([iw])], text_width='300px')
z.layout.width = '95%'
z

In [10]:
display(widgets.HTML("""<HR><H4>If you didn't find what you were looking for,
    try increasing the number of responses or rehrasing your question"""))
display(slider)
widgets.HBox([widgets.VBox([textbox])])

HTML(value="<HR><H4>If you didn't find what you were looking for,\n    try increasing the number of responses …

IntSlider(value=5, continuous_update=False, description='Responses', max=25, min=1)